In [ ]:
import nltk as nltk
import pandas as pd
import matplotlib.pyplot as plt
import re

In [ ]:
data=pd.read_csv("SMSSpamCollection.csv" , sep="\t" , header=None)

In [ ]:
data.columns=["label" , "body_text"]
data.head(5)

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.label.value_counts()

In [ ]:
data.label.value_counts(normalize=True).plot.pie()
plt.show()

3. Data Preparation-Pre-processing text data

2. تنظيف النصوص (Text Cleaning)

تحويل الحروف لصيغة موحدة (Lowercasing).

إزالة الأرقام (لو مش مهمة).

إزالة الرموز الخاصة (punctuations, emojis لو مش محتاجها).

إزالة المسافات الزائدة.

🔹 3. إزالة الكلمات عديمة القيمة (Stopwords Removal)

كلمات زي "the, is, in, من، على" غالبًا مش بتضيف معنى كبير.

🔹 4. تجزئة النص (Tokenization)

تقسم النص إلى كلمات أو جُمل.

مثال: "I love NLP" → ["I", "love", "NLP"].

🔹 5. تحويل الكلمات لصيغها الجذرية أو الأساسية

Stemming: إرجاع الكلمة لجذرها (e.g., "playing" → "play").

Lemmatization: إرجاع الكلمة لصيغتها الصحيحة (e.g., "better" → "good").

🔹 6. تحويل النص لتمثيل عددي (Feature Extraction)

عشان الموديل يفهم، النص لازم يتحول لأرقام:

Bag of Words (BoW)

TF-IDF

Word Embeddings (Word2Vec, GloVe, FastText)

Transformers Embeddings (BERT, GPT …)

🔹 7. التعامل مع النصوص الغير متوازنة أو طويلة

Padding/Truncating: عشان النصوص يكون لها نفس الطول.

Oversampling/Undersampling: لو عندك كلاس موزون بشكل سيء.

🔹 8. التقسيم (Train-Test Split)

تقسم البيانات لـ:

تدريب (Train)

تحقق (Validation)

اختبار (Test)

In [ ]:
import string
string.punctuation

In [ ]:
#list comprhansion 
#lambda function
#adding join to join chars into words 
def remove_punct(text):
    
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

data['body_text_nopunc'] = data['body_text'].apply(lambda x: remove_punct(x))

data.head()

In [ ]:
#add lower to teh remove_punc function

#list comprhansion 
#lambda function
#adding join to join chars into words 
def remove_punct(text):
    
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

data['body_text_nopunc'] = data['body_text'].apply(lambda x: remove_punct(x.lower()))

data.head()

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
import re
#\W+ regex, indicates that it will split wherever it sees one or more non-word characters.
#So that'll split on white spaces, special characters, anything like that.

def tokenize(text):
    tokens = word_tokenize(text)
    return tokens
data['body_text_tokenized'] = data['body_text_nopunc'].apply(lambda x: tokenize(x))

data.head()

3.4 Remove stopwords
Stopwords are common words that are present in the text but generally do not contribute to the meaning of a sentence. They hold almost no importance for the purposes of information retrieval and natural language processing. They can safely be ignored without sacrificing the meaning of the sentence. For example – ‘the’ and ‘a’.

Stop Words: A stop word is a commonly used word (such as “the”, “a”, “an”, “in”) that a search engine has been programmed to ignore, both when indexing entries for searching and when retrieving them as the result of a search query.

The NLTK package has a separate package of stop words that can be downloaded. NLTK has stop words more than 16 languages which can be downloaded and used. Once it is downloaded, it can be passed as an argument indicating it to ignore these words.

import nltk  from nltk.corpus 
import stopwords  set(stopwords.words('english')) 

In [ ]:
import nltk
from nltk.corpus import stopwords
stopwords_En = nltk.corpus.stopwords.words('english')
stopwords_En

In [ ]:
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopwords_En]
    return text

data['body_text_nostop'] = data['body_text_tokenized'].apply(lambda x: remove_stopwords(x))

data.head()

🔹 1. Stemming

هو عملية قص أو تقطيع الكلمة عشان نوصل لجذرها (Root)، لكن بشكل بسيط وسريع من غير ما يهتم بالمعنى اللغوي.

النتيجة ساعات بتكون كلمة غير موجودة فعليًا في القاموس.

✅ مثال:

"Studies" → "Studi"

"Studying" → "Study" أو أحيانًا "Studi"

"Better" → "Better" (ممكن يسيبها زي ما هي لأن الـ stemmer مش فاهم المعنى)

📌 أداة مشهورة: Porter Stemmer أو Snowball Stemmer في مكتبة NLTK.

ميزة: أسرع.

عيب: ممكن تطلع نتائج مش دقيقة أو كلمات مالهاش معنى.

🔹 2. Lemmatization

بتعتمد على القاموس (Dictionary) + القواعد اللغوية (Morphology).

بتديك الجذر الصحيح للكلمة اللي موجود فعلًا في القاموس.

بتاخد في اعتبارها نوع الكلمة (POS Tag: verb, noun, adjective, ...).

✅ مثال:

"Studies" → "Study" (اسم أو فعل)

"Studying" → "Study"

"Better" → "Good" (لأنها فاهمة إن "better" هي صيغة تفضيل من "good")

📌 أداة مشهورة: WordNet Lemmatizer في NLTK أو spacy Lemmatizer.

ميزة: أدق.

عيب: أبطأ من stemming لأنها محتاجة قاعدة بيانات لغوية.

🔑 الخلاصة:

لو عايز سرعة (زي في search engines) → استخدم Stemming.

لو عايز دقة ومعنى صحيح (زي NLP applications أو chatbots) → استخدم Lemmatization.

In [ ]:
import nltk
ps = nltk.PorterStemmer()
def stemming(tokenized_text):
    text = [ps.stem(word) for word in tokenized_text]
    return text

data['body_text_stemmed'] = data['body_text_nostop'].apply(lambda x: stemming(x))

data.head()

In [ ]:
import nltk

wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()

In [ ]:
def lemmatizing(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

data['body_text_lemmatized'] = data['body_text_nostop'].apply(lambda x: lemmatizing(x))

data.head(5)

lemmatizer=>ادق ف المعني 

In [ ]:

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()

def clean_text_lemma(text):
    # 1. إزالة علامات الترقيم وتحويل لحروف صغيرة
    text = "".join([char.lower() for char in text if char not in string.punctuation])
    # 2. تقسيم النص لكلمات
    tokens = word_tokenize(text)
    # 3. إزالة stopwords + Lemmatization
    text = " ".join([lemmatizer.lemmatize(word) for word in tokens if word not in stopwords_En])
    return text
data=data[['label','body_text']]
data['cleaned_text'] = data['body_text'].apply(lambda x: clean_text_lemma(x))
data

In [ ]:

### Create function to remove punctuation, tokenize, remove stopwords, and stem=اسرع

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
     #tokens = re.split('\W+', text)
    tokens = word_tokenize(text)
    text = " ".join([ps.stem(word) for word in tokens if word not in stopwords_En])
    return text
data=data[['label','body_text']]
data['cleaned_text'] = data['body_text'].apply(lambda x: clean_text(x))

In [ ]:
data

🔑 الخلاصة:

لو مشروع بسيط (تصنيف Spam/Not Spam مثلاً) → استخدم TF-IDF.

لو مشروع عايز معنى أعمق (Sentiment Analysis, Chatbot, Translation) → استخدم Word Embeddings.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
#tfidf = TfidfVectorizer(min_df=1)
tfidf = TfidfVectorizer( ngram_range=(2,2))
features_tfidf = tfidf.fit_transform(data['cleaned_text'])
print(features_tfidf.shape)
print('Sparse Matrix :\n', features_tfidf)
features_tfidf = pd.DataFrame(features_tfidf.toarray())
features_tfidf.columns = tfidf.get_feature_names_out()
features_tfidf